# 5.12 DenseNet


相比于RenseNet将输出相加，DenseNet将输出在通道维上连结，所以又叫做稠密连接。

DenseNet主要模块使稠密块和过度层：

+ 稠密块定义了网络输入和输出的连结

+ 过渡层用来控制通道数，使之不过大。

DenseNet网络的结构图如下：
<img src="https://gitee.com/changyv/md-pic/raw/master/20210318163756.png" alt="image-20210318163555548" style="zoom: 75%;" />

定义稠密快：

In [1]:
import time 
import torch 
from torch import nn,optim 
import torch.nn.functional as F 

import sys 
sys.path.append('..')
import d2l_pytorch as d2l 

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# 定义卷积层，构成稠密块
def conv_block(in_channels,out_channels):
    blk=nn.Sequential(
        nn.BatchNorm2d(in_channels),
        nn.ReLU(),
        nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1)
    )
    return blk

In [6]:
# 定义稠密快
class dense_block(nn.Module):
    def __init__(self,num_convs,in_channels,out_channels):
        super(dense_block,self).__init__()
        net=[]
        for i in range(num_convs):
            in_c=in_channels + i * out_channels
            net.append(conv_block(in_c,out_channels))
        # 网络
        self.net=nn.ModuleList(net)
        self.out_channels=in_channels +num_convs*out_channels

    def forward(self,x):
        for blk in self.net:
            y=blk(x)
            # 在通道上进行连结
            x=torch.cat((x,y),dim=1)
        return x

In [7]:
# test
blk=dense_block(2,3,10)

x=torch.rand(4,3,8,8)

y=blk(x)
y.shape

torch.Size([4, 23, 8, 8])

过渡层

由于每个稠密块都会带来通道数的增加，使用过多则会带来过于复杂的模型，

过渡层用来控制模型复杂度。它通过1×1卷积层来减小通道数，并使用步幅为2的平均池化层减半高和宽，从而进一步降低模型复杂度。


In [10]:
def transition_block(in_channels,out_channels):
    blk=nn.Sequential(
        nn.BatchNorm2d(in_channels),
        nn.ReLU(),
        nn.Conv2d(in_channels,out_channels,kernel_size=1),
        nn.AvgPool2d(kernel_size=2,stride=2)
    )
    return blk

In [11]:
# test
# 将上述例子中的通道数23减为10

blk=transition_block(23,10)

blk(y).shape

torch.Size([4, 10, 4, 4])

## 构建DenseNet模型
